In [ ]:
import numpy as np
import random
from itertools import product
import pandas as pd


class Frota:
    def __init__(self, Periodo, TamanhoFrota, Demanda):
        if len(Demanda) < Periodo:
            Demanda = Demanda + \
                [random.randint(0, TamanhoFrota)
                 for i in range(Periodo-len(Demanda))]
        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        # self.Parametros = {}
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: self.ProbQuebra_0 / \
            (self.ProbQuebra_0+(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x))
        self.EstadosAcoes = {}
        # self.Acoes = {}
        self.Politica = {}
        self.ConstroeEstadosAcoes()

    def ConstroeEstadosAcoes(self):
        aux = []
        for i in range(self.Periodo+1):
            for j in range(i+1):
                aux.append((i, j))

        estados = list(product(*[aux for i in range(self.TamanhoFrota)]))

        acoes = list(product(*[[0, 1, 2] for i in range(self.TamanhoFrota)]))
        self.Estados = estados
        # self.Acoes = acoes
        estado_acoes = {}
        for est in estados:
            vmax = max([e[0] for e in est])
            for i in range(vmax, self.Periodo):
                aux = []
                # estado_acoes.append((*est,i))
                for ac in acoes:
                    cnt = sum([a == 0 for a in ac])
                    if cnt >= self.Demanda[i]:
                        if i < self.Periodo-1:
                            aux.append(ac)
                        else:
                            aux.append(tuple([-1]*self.TamanhoFrota))
                estado_acoes[(*est, i)] = aux
        estado_acoes[(*tuple([(-1, -1)]*self.TamanhoFrota), i+1)
                     ] = [tuple([-1]*self.TamanhoFrota)]
        self.EstadosAcoes = estado_acoes
        return estado_acoes

    def Recompensa(self, est):
        tvida = est[0]

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)

        # define o valor da manutenção se der defeito
        valor_defeito = -2*valor_manutencao/valor_equipamento

        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op, res_a0_de = 0.01/valor_equipamento, valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao/valor_equipamento

        # Cálculo das recompensas na troca de equipamento
        res_a2 = -(1 - 0.8*valor_equipamento)

        res = [res_a0_op, res_a0_de, res_a1, res_a2]
        if est[0] == -1:
            res = [0, 0, 0, 0]
        return res

    def ProximoEstado(self, estado, acao):
        NovosEstados = {}
        NovoEstado = [[]]*self.TamanhoFrota
        Recompensa = [[]]*self.TamanhoFrota
        Probabilidades = [[]]*self.TamanhoFrota
        for i, a in enumerate(acao):
            rcp_aux = self.Recompensa(estado[i])
            if a == 0:
                NovoEstado[i] = [(estado[i][0]+1, estado[i][1]+1),(estado[i][0]+1, 0)]
                prob_quebrar = self.ProbQuebraFcn(estado[i][1])
                Probabilidades[i] = [1-prob_quebrar, prob_quebrar]
                Recompensa[i] = [rcp_aux[0], rcp_aux[1]]
            if a == 1:
                NovoEstado[i] = [(estado[i][0]+1, 0)]
                Probabilidades[i] = [1]
                Recompensa[i] = [rcp_aux[2]]
            if a == 2:
                NovoEstado[i] = [(0, 0)]
                Probabilidades[i] = [1]
                Recompensa[i] = [rcp_aux[3]]
            if a == -1:
                NovoEstado[i] = [(-1, -1)]
                Probabilidades[i] = [1]
                Recompensa[i] = [0]

        # print(NovoEstado)

        probs = list(product(*Probabilidades))
        recs = list(product(*Recompensa))
        probs = [np.prod(pr) for pr in probs]
        recs = [np.sum(rc) for rc in recs]
        if self.Periodo > estado[-1]:
            tmp = estado[-1]+1
        else:
            tmp = self.Periodo

        NovosEstados["estados"] = [(*x, tmp)
                                   for x in list(product(*NovoEstado))]
        NovosEstados["recompensas"] = recs
        NovosEstados["probabilidades"] = probs

        return NovosEstados

    def ValueIteration(self, tol=1e-5, sims=500):

        lista_estados = list(self.EstadosAcoes.keys())
        # est0 = lista_estados[0]
        if len(self.Politica) == 0:
            for est in lista_estados:
                aux = [0]*len(self.EstadosAcoes[est])
                # aux[random.choice(range(len(aux)))] = 1
                aux[0] = 1
                self.Politica[est] = {ac: x for ac,x in zip(self.EstadosAcoes[est], aux)}

        VN = {v: 0.05 for v in lista_estados}
        VN[(*tuple([(-1, -1)]*self.TamanhoFrota), self.Periodo)] = 0

        PLN = {k: np.argmax(v) for k, v in self.Politica.items()}

        M, P = {}, {}
        M[0] = np.array(list(VN.values()))
        P[0] = {k: v for k, v in PLN.items()}

        cnt, nor = 0, 10
        val_norm = []
        while nor > tol and cnt <= sims:

            # PL = {k:v for k,v in PLN.items()}
            PLN = {}

            V = {k: v for k, v in VN.items()}
            VN = {}

            for est in lista_estados:
                v_aux = []
                for idx, ac in enumerate(self.EstadosAcoes[est]):
                    next = self.ProximoEstado(est, ac)
                    nest = next["estados"]
                    probs = next["probabilidades"]
                    recs = next["recompensas"]
                    v_aux2 = 0
                    for j_e in range(len(nest)):
                        v_aux2 = v_aux2 + probs[j_e]*(recs[j_e]+V[nest[j_e]])
                    v_aux.append(v_aux2)

                VN[est] = np.max(v_aux)
                PLN[est] = np.argmax(v_aux)

            cnt = cnt+1
            nor = np.linalg.norm(
                np.array(list(V.values()))-np.array(list(VN.values())))
            val_norm.append(nor)
            print(
                f"sim: {cnt:2d}, norma: {nor:5.2f}, condicao: {nor>tol}", end='\r')
            M[cnt % 2] = np.array(list(VN.values()))
            P[cnt % 2] = {k: v for k, v in PLN.items()}

        # PoliticaOtima = {est:self.EstadosAcoes[est][idx] for est,idx in P[1].items()}
        self.Politica = {
            est: self.EstadosAcoes[est][idx] for est, idx in P[1].items()}

        return {"politica": self.Politica, "valor": M[1], "norma": val_norm}

In [ ]:
mdl = Frota(12,3,[2,2,2,2,2,1,2,2])

In [ ]:
res = mdl.ValueIteration()

In [ ]:
est0 = ((0,0),(0,0),(0,0),0)
ac = res["politica"][est0]
mdl.ProximoEstado(est0,ac)
est1 = ((1, 1), (1, 1), (1, 1), 1)
ac = res["politica"][est1]
mdl.ProximoEstado(est1,ac)
est2 = ((2, 2), (2, 2), (2, 2), 2)
ac = res["politica"][est2]
mdl.ProximoEstado(est2,ac)
est3 = ((3, 3), (3, 3), (3, 3), 3)
ac = res["politica"][est3]
mdl.ProximoEstado(est3,ac)
est4 = ((4, 4), (4, 4), (4, 4), 4)
ac = res["politica"][est4]
mdl.ProximoEstado(est4,ac)
est5 = ((5, 5), (5, 5), (5, 5), 5)
ac = res["politica"][est5]
mdl.ProximoEstado(est5,ac)
est6 = ((6, 6), (6, 0), (6, 0), 6)
ac = res["politica"][est6]
mdl.ProximoEstado(est6,ac)
est7=((7, 7), (7, 1), (7, 1), 7)
ac = res["politica"][est7]
mdl.ProximoEstado(est7,ac)

In [ ]:
sem = 7
res["politica"][((sem,sem),(sem,sem),(sem,sem),sem)]

In [ ]:
{est:ac for est,ac in res["politica"].items() if max(ac)>0}

In [ ]:
res["politica"][1]